In [1]:
from src.agents import Volatile
import multiprocessing as mp

In [2]:
def run_simulation(instance, hops):
    for j in range(hops):
        instance.migrate(2.989e-26)

In [6]:
volatile_1 = Volatile()
volatile_2 = Volatile()
volatile_3 = Volatile()
volatile_4 = Volatile()
volatile_5 = Volatile()
volatile_6 = Volatile()

run_simulation(volatile_1, 5000)

[2.34504159 0.69024936 5.71900485 ... 3.16583452 5.68658692 3.06447789]
[2.34504159 0.69024936 5.71900485 ... 3.16583452 5.68658692 3.06447789]
